In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/china-airlines/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'china airlines'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']


# Save to CSV
df.to_csv('Data/china_airline.csv',index=False)

In [4]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,china airlines,Tim Gould,2020-04-05,✅ Trip Verified | Hong Kong to Brisbane via T...,Business,Premium Economy,1,1,1,1,1,1,1,no
1,china airlines,Joshua Lok,2020-03-23,✅ Trip Verified | Jakarta to Vancouver via Ta...,Family Leisure,First Class,10,4,5,5,5,5,5,yes
2,china airlines,Julie Ross,2020-03-17,✅ Trip Verified | Myself my mother and my hus...,Family Leisure,Economy Class,1,3,1,1,1,2,3,no
3,china airlines,J Francisco,2020-03-11,✅ Trip Verified | They have no regard for con...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
4,china airlines,Stephan Stegner,2020-02-18,✅ Trip Verified | The check-in in this case w...,Solo Leisure,Economy Class,8,4,5,5,5,4,5,yes


In [5]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,china airlines,S Meaden,2018-01-13,✅ Trip Verified | Rome to Auckland via Taipei...,Couple Leisure,Economy Class,2,1,1,1,2,3,1,no
96,china airlines,E Lee,2018-01-10,✅ Trip Verified | Brisbane to Nanjing via Tai...,Family Leisure,Economy Class,1,1,1,2,2,1,1,no
97,china airlines,Shauna Whelan,2018-01-07,✅ Trip Verified | Amsterdam to Sydney via Taip...,Solo Leisure,Economy Class,1,2,1,NaN,NaN,1,2,no
98,china airlines,T Wen,2018-01-02,✅ Trip Verified | Taipei to Singapore. Cabin ...,Family Leisure,Economy Class,7,5,3,4,2,4,4,yes
99,china airlines,Michelle Valencia,2017-12-30,✅ Trip Verified | Singapore to Surabaya. The ...,Family Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
